# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II - Data Modelling

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Setup code

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    q = """
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor': 1}
    """
    session.execute(q)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Three questions with three corresponding table designs

### 1. Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Return every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'




---------------------------------------
### 1. Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
query1 = """
SELECT artist, song, length FROM events_by_sessionid_iteminsession WHERE session_id = 338 and item_in_session = 4
"""

In [9]:
q = """
CREATE TABLE IF NOT EXISTS events_by_sessionid_iteminsession 
(
    session_id int,
    item_in_session int,
    artist text,
    length double,
    song text,
    PRIMARY KEY (session_id, item_in_session)
)
"""
try:
    session.execute(q)
except Exception as e:
    print(e)

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO events_by_sessionid_iteminsession 
        (
            session_id, 
            item_in_session, 
            artist, 
            length, 
            song
        )"""
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, 
                        (int(line[8]),
                         int(line[3]),
                         line[0],
                         float(line[5]),
                         line[9]
                        ))

#### Data verification

In [11]:
r = session.execute(query1)
for row in r:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.3073)


---------------------------------------
### 2. Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
query2="""
SELECT artist, song, first_name, last_name, item_in_session 
FROM events_by_userid_sessionid_iteminsession 
WHERE user_id = 10 AND session_id = 182 
"""

In [13]:
q = """
CREATE TABLE IF NOT EXISTS events_by_userid_sessionid_iteminsession 
(
    user_id int,
    session_id int,
    item_in_session int,
    artist text,
    first_name text,
    last_name text,    
    song text,
    PRIMARY KEY (user_id, session_id, item_in_session)
) 
"""
try:
    session.execute(q)
except Exception as e:
    print(e)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = '''INSERT INTO events_by_userid_sessionid_iteminsession 
        (
            user_id, 
            session_id, 
            item_in_session, 
            artist, 
            first_name, 
            last_name,  
            song
        )'''
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, 
                        (int(line[10]),
                         int(line[8]),
                         int(line[3]),
                         line[0],
                         line[1],
                         line[4],
                         line[9]
                        ))

In [15]:
r = session.execute(query2)
for row in r:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz', item_in_session=0)
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz', item_in_session=1)
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz', item_in_session=2)
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz', item_in_session=3)


---------------------------------------
### 3. Return every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

In [16]:
query3 ="""
SELECT first_name, last_name 
FROM events_by_song_userid 
WHERE song='All Hands Against His Own'
"""             

In [17]:
q = """
CREATE TABLE IF NOT EXISTS events_by_song_userid 
(
    song text,
    user_id int,
    first_name text,
    last_name text,  
    PRIMARY KEY (song, user_id)
) 
"""
try:
    session.execute(q)
except Exception as e:
    print(e)

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = '''INSERT INTO events_by_song_userid 
        (
            song, 
            user_id, 
            first_name, 
            last_name
        )'''
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, 
                        (line[9],
                         int(line[10]),
                         line[1],
                         line[4]
                        ))

In [19]:
r = session.execute(query3)
for row in r:
    print(row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [20]:
session.execute("DROP TABLE IF EXISTS events_by_sessionid_iteminsession")
session.execute("DROP TABLE IF EXISTS events_by_userid_sessionid_iteminsession")
session.execute("DROP TABLE IF EXISTS events_by_firstname_lastname_song")

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()